[![Jupyter Notebook](https://img.shields.io/badge/Jupyter%20Notebook-orange)](https://github.com/laminlabs/lamin-usecases/blob/main/docs/celltypist.ipynb)

# Manage a cell type registry

Cell types classify cells based on public and private knowledge from studying transcription, morphology, function & other properties.
Established cell types have well-characterized markers and properties; however, cell subtypes and states are continuously being discovered, refined and better understood.

In this notebook, we manage an immune cell type registry from [CellTypist](https://www.celltypist.org), a computational tool used for cell type classification in scRNA-seq data. We'll walk you through the following steps:

1. Create a cell type registry seeded from cell types supported by CellTypist.
2. Use CellTypist to classify cell types of a previously unannotated dataset and track the dataset with LaminDB.
3. Demonstrate how datasets are queryable by cell types using LaminDB.

## Setup

To run this notebook, you need to load a LaminDB instance that has the `bionty` schema mounted.

Here, we'll create a test instance (skip if you'd like to run it using your own instance):

In [ ]:
!lamin init --storage ./celltypist --schema bionty

In [ ]:
# filter warnings from celltypist
import warnings

warnings.filterwarnings("ignore", message=".*The 'nopython' keyword.*")

In [ ]:
import lamindb as ln
import lnschema_bionty as lb
import celltypist
import pandas as pd

lb.settings.species = "human"  # globally set species

In [ ]:
ln.track()

For a start, let's take a look at the public Cell Ontology.

In [ ]:
celltype_bt = lb.CellType.bionty()  # equals to bionty.CellType()

In [ ]:
celltype_bt

## Create an in-house registry of CellTypist terms

### Fetch CellTypist's immune cell encyclopedia

As a first step we will read in CellTypist's immune cell encyclopedia

In [ ]:
description = "CellTypist Pan Immune Atlas v2: basic cell type information"
celltypist_source_v2_url = "https://github.com/Teichlab/celltypist_wiki/raw/main/atlases/Pan_Immune_CellTypist/v2/tables/Basic_celltype_information.xlsx"

# our source data
celltypist_file = ln.File.filter(description=description).one_or_none()

if celltypist_file is None:
    celltypist_df = pd.read_excel(celltypist_source_v2_url)
    celltypist_file = ln.File(celltypist_df).save()
else:
    celltypist_df = celltypist_file.load().head()

It provides an `ontology_id` of the public Cell Ontology for the majority of records.

In [ ]:
celltypist_df.head()

The "Cell Ontology ID" is associated with multiple "Low-hierarchy cell types":

In [ ]:
celltypist_df.set_index(["Cell Ontology ID", "Low-hierarchy cell types"]).head(10)

### Validate terms with the public Cell Ontology

For any cell type record that can be validated against the public Cell Ontology, we'd like to ensure that it's actually validated.

This will avoid that we'll refer to the same cell type with different identifiers.

Let's see how well the Cell Typist reference data can be validated.

All Celltypist labeled ontology IDs are validated using the public Cell Ontology:

In [ ]:
celltype_bt.inspect(celltypist_df["Cell Ontology ID"], celltype_bt.ontology_id);

However, when inspecting the names, most of them don't validate:

In [ ]:
celltype_bt.inspect(celltypist_df["Low-hierarchy cell types"], celltype_bt.name);

A search tells us that terms that are named in plural in Cell Typist occur with a name in singular in the Cell Ontology: 

In [ ]:
celltypist_df["Low-hierarchy cell types"][0]

In [ ]:
celltype_bt.search(celltypist_df["Low-hierarchy cell types"][0]).head(2)

Let's try to strip `"s"` and inspect if more names are now validated. Yes, there are!

In [ ]:
celltype_bt.inspect(
    [i.rstrip("s") for i in celltypist_df["Low-hierarchy cell types"]],
    celltype_bt.name,
);

Every "low-hierarchy cell type" has an ontology id and most "high-hierarchy cell types" also appear as "low-hierarchy cell types" in the Cell Typist table. Four, however, don't, and therefore don't have an ontology ID.

In [ ]:
high_terms = celltypist_df["High-hierarchy cell types"].unique()
low_terms = celltypist_df["Low-hierarchy cell types"].unique()

high_terms_nonval = set(high_terms).difference(low_terms)
high_terms_nonval

### Register CellTypist records in LaminDB

Let's first add the "High-hierarchy cell types" as a column `"parent"`.

This enables LaminDB to populate the `parents` and `children` fields, which will enable you to query for hierarchical relationships.

In [ ]:
celltypist_df["parent"] = celltypist_df.pop("High-hierarchy cell types")

# if high and low terms are the same, no parents
celltypist_df.loc[
    (celltypist_df["parent"] == celltypist_df["Low-hierarchy cell types"]), "parent"
] = None

# rename columns, drop markers
celltypist_df.drop(columns=["Curated markers"], inplace=True)
celltypist_df.rename(
    columns={"Low-hierarchy cell types": "name", "Cell Ontology ID": "ontology_id"},
    inplace=True,
)
celltypist_df.columns = celltypist_df.columns.str.lower()

In [ ]:
celltypist_df.head(2)

Now, let's create records from the public ontology:

In [ ]:
public_records = lb.CellType.from_values(
    celltypist_df.ontology_id, lb.CellType.ontology_id
)

Let's now amend public ontology records so that they maintain additional annotations that Cell Typist might have.

In [ ]:
records_names = {}
public_records_dict = {r.ontology_id: r for r in public_records}

for _, row in celltypist_df.iterrows():
    name = row["name"]
    ontology_id = row["ontology_id"]
    public_record = public_records_dict[ontology_id]

    # if both name and ontology_id match public record, use public record
    if name.lower() == public_record.name.lower():
        records_names[name] = public_record
        continue
    else:  # when ontology_id matches the public record and name doesn't match
        # if singular form of the Celltypist name matches public name
        if name.lower().rstrip("s") == public_record.name.lower():
            # add the Celltypist name to the synonyms of the public ontology record
            public_record.add_synonym(name)
            records_names[name] = public_record
            continue
        if public_record.synonyms is not None:
            synonyms = [s.lower() for s in public_record.synonyms.split("|")]
            # if any of the public matches celltypist name
            if any(
                [
                    i.lower() in {name.lower(), name.lower().rstrip("s")}
                    for i in synonyms
                ]
            ):
                # add the Celltypist name to the synonyms of the public ontology record
                public_record.add_synonym(name)
                records_names[name] = public_record
                continue

        # create a record only based on Celltypist metadata
        records_names[name] = lb.CellType(
            name=name, ontology_id=ontology_id, description=row.description
        )

You can see certain records are created by adding the Celltypist name to the synonyms of the public record:

In [ ]:
records_names["GMP"]

Other records are created based on Celltypist metadata:

In [ ]:
records_names["Age-associated B cells"]

Let's save them to our database:

In [ ]:
records = set(records_names.values())

ln.save(records)

### Add parent-child relationship of the records from Celltypist

We still need to add the renaming 4 High hierarchy terms:

In [ ]:
list(high_terms_nonval)

Let's get the top hits from a search:

In [ ]:
for term in list(high_terms_nonval):
    print(f"Term: {term}")
    display(celltype_bt.search(term).head(1))

So we decide to:

- Add the "T cells" to the synonyms of the public "T cell" record
- Create the remaining 3 terms only using their names (we think "B cell flow" shouldn't be identified with "B cell")

In [ ]:
for name in high_terms_nonval:
    if name == "T cells":
        record = lb.CellType.from_bionty(name="T cell")
        record.add_synonym(name)
        record.save()
    else:
        record = lb.CellType(name=name)
        record.save()
    records_names[name] = record

Now let's add the parent records:

In [ ]:
for _, row in celltypist_df.iterrows():
    record = records_names[row["name"]]
    if row["parent"] is not None:
        parent_record = records_names[row["parent"]]
        record.parents.add(parent_record)

## Access the in-house CellType registry

The previously added CellTypist ontology registry is now available in LaminDB.
To retrieve the full ontology table as a Pandas DataFrame we can use `.filter`:

In [ ]:
lb.CellType.filter().df()

This enables us to look for cell types by creating a lookup object from our new `CellType` registry.

In [ ]:
db_lookup = lb.CellType.lookup()

In [ ]:
db_lookup.memory_b_cell

See cell type hierarchy:

In [ ]:
db_lookup.memory_b_cell.view_parents()

Access parents of a record:

In [ ]:
db_lookup.memory_b_cell.parents.list()

## Annotate a dataset with cell types using CellTypist

### Annotate cell types predicted with CellTypist

We now demonstrate how simple it is to predict and add cell types to LaminDB with CellTypist.
Our dataset of choice is a simple sample dataset together with a sample model.

In [ ]:
input_file = celltypist.samples.get_sample_csv()
input_file

In [ ]:
predictions = celltypist.annotate(
    input_file, model="Immune_All_Low.pkl", majority_voting=True
)

Now that we've predicted all cell types we create an [Anndata](https://anndata.readthedocs.io/en/latest) object that we will eventually track with LaminDB.

In [ ]:
adata_annotated = predictions.to_adata()

In [ ]:
adata_annotated.obs

Validate cell types:

In [ ]:
celltypes = lb.CellType.from_values(
    adata_annotated.obs.predicted_labels, lb.CellType.name
)

### Track the annotated dataset in LaminDB

#### Register features

In [ ]:
features = ln.Feature.from_df(adata_annotated.obs)
ln.save(features)

Create a file record of the AnnData object.
We further define a name of the dataset for clarity that can also be queried for.

In [ ]:
file_annotated = ln.File.from_anndata(
    adata_annotated, description="Examplary CellTypist file", var_ref=lb.Gene.symbol
)

In [ ]:
file_annotated.save()

In [ ]:
file_annotated.add_labels(celltypes, feature="predicted_labels")

In [ ]:
file_annotated.describe()

In [ ]:
file_annotated.view_flow()

Now we can query a file by a specific cell type:

In [ ]:
ln.File.filter(cell_types=db_lookup.tcm_naive_helper_t_cells).df()

Or track in which notebook the file is annotated by celltypist:

In [ ]:
ln.Transform.filter(files__description__icontains="CellTypist").df()

In [ ]:
# clean up test instance
!lamin delete --force celltypist
!rm -r ./celltypist